Hey, I'm Jobert Gutierrez and hereafter you'll find the logic and code used to answer the first assignment in the program Machine Learning Operations Zoomcamp offered by Data Talks Club.

# Module 1 Homework: General Overview and foundations
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Question 1. Downloading the data
We'll use the same [NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

## Answer:

I downloaded the data using this code:

In [ ]:
# For January 2023:
python -m wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

# For February 2023:
python -m wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet


Then, I read the January file as:

In [119]:
# Calling the libraries used:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [120]:
df = pd.read_parquet(r'C:/Users/jober/OneDrive/Desktop/mlops-hw2025/Homework_week01/yellow_tripdata_2023-01.parquet')

The number of columns is identified using the function:

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Question 2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

## Answer:

The duration of the trips are calculated:

In [122]:
# The amount of time for each ride is:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

# Expressed in mutes is:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

# The stard deviation of the ride's duration is: 
df.duration.std()

42.594351241920904

## Question 3. Dropping outliers
Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

## Answer:

The distribution of duration variables are:

In [123]:
# The number of registries before removing outliers is:
r_before = len(df)
print(f'the number of rows with outliers is: {r_before}')

# Removing outliers:
df = df[(df.duration >= 1) & (df.duration <= 60)]

# The number of rows without outliers is:
r_after = len(df)
print(f'the number of rows without outliers is: {r_after}')

the number of rows with outliers is: 3066766
the number of rows without outliers is: 3009173


In [124]:
fraction = r_after/r_before
print(f'Then, the fraction of the records left after removing the outliers is: {fraction}') 


Then, the fraction of the records left after removing the outliers is: 0.9812202822125979


## Question 4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

## Answer:

The dimensionality is calculated as follows:

In [125]:
# The categorical variables selected are:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
strings = ['VendorID', 'RatecodeID']

df[categorical] = df[categorical].astype(str)
df[strings] = df[strings].astype(str)

# Using the variables indicated, the train dictionary is:
train_dicts = df[categorical].to_dict(orient='records')

# The feature matrix is calculated as:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# The shape of this matrix is:
X_train.shape

(3009173, 515)

The number of columns of this matrix is __515 columns__.

## Question 5. Training a model
Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

## Answer:

The RSME is calculated as follows:

In [126]:
# The value of the target is:
target = 'duration'
y_train = df[target].values

# The plain linear model to train is:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [127]:
# The predicted target value for duration is:
y_pred = lr.predict(X_train)

# The deviation for duration is:
rsme_train = root_mean_squared_error(y_train, y_pred)
print(f'The rmse for training dataset is: {rsme_train}')

The rmse for training dataset is: 7.649261927651774


## Question 6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

## Answer:

The RSME is calculated for february 2023 as follows:

In [128]:
# I'll use the following function:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [129]:
# Using the fuction created I can read the data as:
file = 'C:/Users/jober/OneDrive/Desktop/mlops-hw2025/Homework_week01/yellow_tripdata_2023-02.parquet'

# The validation dataframe is:
df_val = read_dataframe(file)

In [130]:
# The categorical variables selected are:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
strings = ['VendorID', 'RatecodeID']

df_val[categorical] = df_val[categorical].astype(str)
df_val[strings] = df_val[strings].astype(str)

# Using the variables indicated, the train dictionary is:
val_dicts = df_val[categorical].to_dict(orient='records')

# The feature matrix is calculated as:
X_val = dv.transform(val_dicts)

In [131]:
# Keeping duration as the target value
y_val = df_val[target].values

In [133]:
# The predicted value for linear model is:
y_pred_2 = lr.predict(X_val)

# The deviation for duration is:
rsme_val = root_mean_squared_error(y_val, y_pred_2)
print(f'The rmse for the validation dataset is: {rsme_val}')

The rmse for the validation dataset is: 7.811819692493332
